In [37]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy import optimize
import math

In [38]:
tickers = [
    'AAPL',
    'MSFT',
    'GOOG',
    'AMZN',
    'BRK-B',
    'V',
    'XOM',
    'UNH',
    'JNJ',
    'NVDA',
]

period = '10y' # Options are 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, and max

interval = '1d' # Options are 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, and 3mo

allow_short = False

In [39]:
fields = []
for i in range(len(tickers)):
    fields.append((tickers[i], 'Close'))

df = yf.download(
    tickers = tickers,
    period = period,
    interval = interval,
    group_by = 'ticker'
)[fields]

daily_returns = df.pct_change().dropna()

yearly_returns = daily_returns.mean() * 252

print(yearly_returns.head())

[*********************100%***********************]  10 of 10 completed
AAPL   Close    0.262292
MSFT   Close    0.255640
GOOG   Close    0.205286
AMZN   Close    0.253801
BRK-B  Close    0.134603
dtype: float64


In [40]:
covariance = daily_returns.cov() * 252

print(covariance.head())

                 AAPL      MSFT      GOOG      AMZN     BRK-B         V  \
                Close     Close     Close     Close     Close     Close   
AAPL  Close  0.082737  0.049483  0.045503  0.048892  0.028163  0.038719   
MSFT  Close  0.049483  0.073711  0.051539  0.053102  0.028868  0.041652   
GOOG  Close  0.045503  0.051539  0.073848  0.056406  0.027309  0.039791   
AMZN  Close  0.048892  0.053102  0.056406  0.106959  0.024431  0.039487   
BRK-B Close  0.028163  0.028868  0.027309  0.024431  0.037378  0.031008   

                  XOM       UNH       JNJ      NVDA  
                Close     Close     Close     Close  
AAPL  Close  0.024785  0.030639  0.018401  0.069779  
MSFT  Close  0.024383  0.032156  0.020296  0.071527  
GOOG  Close  0.024978  0.029168  0.017981  0.066064  
AMZN  Close  0.019817  0.025896  0.015814  0.073578  
BRK-B Close  0.030754  0.026974  0.019114  0.035884  


In [41]:
def portfolio_var(w, cov):
    w = np.matrix(w)
    cov = np.matrix(cov)
    result = w * cov * w.T
    return result

def portfolio_returns(w):
    return np.sum(w * yearly_returns)

cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

bounds = [()]
if allow_short:
    bounds = tuple((-np.inf, np.inf) for i in tickers)
else:
    bounds = tuple((0, 1) for i in tickers)

w0 = [1 / len(tickers)] * len(tickers)

In [42]:
abs_min_variance = optimize.minimize(
    fun = portfolio_var,
    x0 = w0,
    args = covariance,
    method = 'SLSQP',
    bounds = bounds,
    constraints = cons
)

print('Absolute Minimum Variance Portfolio')
for i in range(len(tickers)):
    print(tickers[i], '=', round(abs_min_variance.x[i] * 100, 3), '%')
print('Expected Annual Returns =', round(portfolio_returns(abs_min_variance.x) * 100, 3), '%')

Absolute Minimum Variance Portfolio
AAPL = 2.153 %
MSFT = 0.0 %
GOOG = 2.769 %
AMZN = 5.647 %
BRK-B = 24.889 %
V = 0.0 %
XOM = 7.936 %
UNH = 2.93 %
JNJ = 53.676 %
NVDA = 0.0 %
Expected Annual Returns = 12.412 %


In [43]:
expected_return = 0.15

cons = (
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
    {'type': 'eq', 'fun': lambda x: portfolio_returns(x) - expected_return}
    )

min_variance = optimize.minimize(
    fun = portfolio_var,
    x0 = w0,
    args = covariance,
    method = 'SLSQP',
    bounds = bounds,
    constraints = cons
)

print('Minimum Variance Portfolio for', expected_return * 100, '% Annual Returns')
for i in range(len(tickers)):
    print(tickers[i], '=', round(min_variance.x[i] * 100, 3), '%')

Minimum Variance Portfolio for 15.0 % Annual Returns
AAPL = 5.344 %
MSFT = 0.0 %
GOOG = 2.217 %
AMZN = 7.161 %
BRK-B = 22.481 %
V = 1.139 %
XOM = 3.255 %
UNH = 10.358 %
JNJ = 46.717 %
NVDA = 1.329 %


In [44]:
risk_free = 0 # Leave as 0 to use 10 Years Treasurey Yield

if risk_free == 0:
    risk_free = yf.Ticker('^TNX').history(period = '5d')['Close']
    risk_free = risk_free.iloc[len(risk_free) - 1]
    risk_free = risk_free / 100

print('Risk-Free Rate =', round(risk_free * 100, 3), '%')

Risk-Free Rate = 3.518 %


In [45]:
def portfolio_sharpe(w, cov):
    a = portfolio_returns(w) - 0
    b = math.sqrt(portfolio_var(w, cov))
    return -a / b

cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

sharpe_ratio = optimize.minimize(
    fun = portfolio_sharpe,
    x0 = w0,
    args = covariance,
    method = 'SLSQP',
    bounds = bounds,
    constraints = cons
)

print('Maximum Sharpe Ratio Portfolio')
for i in range(len(tickers)):
    print(tickers[i], '=', round(sharpe_ratio.x[i] * 100, 3), '%')
print('Expected Annual Returns =', round(portfolio_returns(sharpe_ratio.x) * 100, 3), '%')
print('Sharpe Ratio =', round(-portfolio_sharpe(sharpe_ratio.x, covariance), 3))

Maximum Sharpe Ratio Portfolio
AAPL = 10.617 %
MSFT = 2.802 %
GOOG = 0.0 %
AMZN = 4.567 %
BRK-B = 0.0 %
V = 0.259 %
XOM = 0.0 %
UNH = 46.188 %
JNJ = 0.0 %
NVDA = 35.566 %
Expected Annual Returns = 34.547 %
Sharpe Ratio = 1.327
